# KNNBasic 알고리즘 기본 코드

KNN 최근접이웃 기반 잠재 요인 협업 필터링

## #01. 준비작업

### [1] 패키지 가져오기

In [1]:
import warnings
warnings.filterwarnings('ignore')

# Intel SKlearn 하드웨어 가속 패치 설정
import sys
if sys.platform == 'win32':
    from sklearnex import patch_sklearn
    patch_sklearn()

from helper.util import *
from helper.plot import *
from helper.analysis import *

from surprise import Reader, Dataset, KNNBasic
from surprise.model_selection import train_test_split, cross_validate, GridSearchCV, RandomizedSearchCV
from surprise.accuracy import rmse, mae

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


### [2] 데이터 가져오기

In [2]:
origin = my_read_excel("https://data.hossam.kr/mldata/movie_ratings.xlsx", 
                    sheet_name='ratings', info=False)

## #02. 데이터 전처리

### [1] Surprise 형식의 데이터로 변환

`사용자 번호, 아이템 번호, 평점` 구조의 데이터를 만족해야 한다.

In [3]:
df = origin.drop('timestamp', axis=1)

# 평점의 분포를 알려준다.
reader = Reader(rating_scale=(0.5, 5.0))

data = Dataset.load_from_df(df, reader)
data

## #03. 추천 모형 구현

### [1] 하이퍼파라미터 튜닝

In [4]:
params = {
    "bsl_options" : {
        "k": [30, 40, 50],          # 최대 근접 이웃 수 (기본값=40)
        "min_k": [1, 2, 3],         # 최소 근접 이웃 수 (기본값=1)
        "method": ["als", "sgd"],
        "n_ephocs": [20, 40]
    }, 
    "sim_options": {
        # 코사인 유사도(msd), 피어슨 유사도(pearson), 피어슨-베이스라인 유사도
        "name": ["msd", "pearson", "pearson_baseline"]
    }
}

# grid = GridSearchCV(KNNBasic, 
#                     param_grid=params, 
#                     measures=['RMSE', 'MAE'], 
#                     cv=5, 
#                     n_jobs=-1)

grid = RandomizedSearchCV(KNNBasic, 
                        param_distributions=params, 
                        measures=['RMSE', 'MAE'], 
                        cv=5, 
                        n_jobs=-1)

grid.fit(data)

### [2] 결과확인

#### (1) 하이퍼파라미터와 확인

In [5]:
grid.cv_results['params']

array([{'bsl_options': {'k': 40, 'min_k': 1, 'method': 'sgd', 'n_ephocs': 40}, 'sim_options': {'name': 'msd', 'user_based': True}},
       {'bsl_options': {'k': 50, 'min_k': 2, 'method': 'sgd', 'n_ephocs': 40}, 'sim_options': {'name': 'pearson_baseline', 'user_based': True}},
       {'bsl_options': {'k': 50, 'min_k': 1, 'method': 'sgd', 'n_ephocs': 40}, 'sim_options': {'name': 'pearson', 'user_based': True}},
       {'bsl_options': {'k': 30, 'min_k': 3, 'method': 'sgd', 'n_ephocs': 40}, 'sim_options': {'name': 'pearson', 'user_based': True}},
       {'bsl_options': {'k': 50, 'min_k': 2, 'method': 'sgd', 'n_ephocs': 40}, 'sim_options': {'name': 'pearson', 'user_based': True}},
       {'bsl_options': {'k': 50, 'min_k': 3, 'method': 'sgd', 'n_ephocs': 20}, 'sim_options': {'name': 'pearson', 'user_based': True}},
       {'bsl_options': {'k': 30, 'min_k': 3, 'method': 'als', 'n_ephocs': 40}, 'sim_options': {'name': 'pearson', 'user_based': True}},
       {'bsl_options': {'k': 40, 'min_k': 1

#### (3) 최적 하이퍼파라미터 확인

In [6]:
grid.best_params

{'rmse': {'bsl_options': {'k': 40,
   'min_k': 1,
   'method': 'sgd',
   'n_ephocs': 40},
  'sim_options': {'name': 'msd', 'user_based': True}},
 'mae': {'bsl_options': {'k': 40, 'min_k': 1, 'method': 'sgd', 'n_ephocs': 40},
  'sim_options': {'name': 'msd', 'user_based': True}}}

#### (4) 최적 추정기

In [7]:
grid.best_estimator

{'rmse': <surprise.prediction_algorithms.knns.KNNBasic at 0x2ed84c7f8d0>,
 'mae': <surprise.prediction_algorithms.knns.KNNBasic at 0x2ed84c7f910>}

#### (5) 최적 추정기를 활용한 컨텐츠 추천

학습을 다시 시켜야 한다.

In [8]:
train, test = train_test_split(data, test_size=0.2, random_state=1234)

In [9]:
estimator = grid.best_estimator['rmse']
estimator.fit(train)
pred = estimator.test(test)
pred[:5]

Computing the msd similarity matrix...
Done computing similarity matrix.


[Prediction(uid=603, iid=3996, r_ui=5.0, est=4.104807871091812, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=199, iid=2912, r_ui=4.0, est=4.085720357369826, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid=416, iid=2716, r_ui=2.0, est=3.5240370370964937, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=589, iid=150, r_ui=4.0, est=3.9733299749945528, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=307, iid=6755, r_ui=4.0, est=3.144829027233701, details={'actual_k': 9, 'was_impossible': False})]

In [10]:
estimator.predict(uid=603, iid=3996)

Prediction(uid=603, iid=3996, r_ui=None, est=4.104807871091812, details={'actual_k': 40, 'was_impossible': False})